In [ ]:
!pip install "transformers>=4.40" peft bitsandbytes accelerate


In [ ]:
!pip install datasets evaluate


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"


In [ ]:
import numpy as np

In [ ]:
import torch
from torch.utils.data import DataLoader


from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

from peft import LoraConfig, get_peft_model, TaskType
import evaluate

In [ ]:
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
!git clone https://github.com/sinhajiya/hf_emotion_classifier.git
import os
import sys
project_root = os.path.abspath("/content/hf_emotion_classifier")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from hf_data import *
from metrics import summarize_evaluation

Cloning into 'hf_emotion_classifier'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 60 (delta 23), reused 19 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (60/60), 1.26 MiB | 3.90 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
ds = emotions()
train = ds['train']
val = ds['validation']
test = ds['test']
l = labels_and_ids()
print(l)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'i2l': {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}, 'l2i': {'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

token_data = ds.map(lambda x: tokenize_batch(x, tokenizer), batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)


In [ ]:
qlora_model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=6,
    id2label=l['i2l'],
    label2id=l['l2i'],
    quantization_config=bnb_config,
    device_map="auto"
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]}



In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"], # Modules to adapt
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)


In [ ]:
peft_model = get_peft_model(qlora_model, lora_config)
peft_model.print_trainable_parameters()


trainable params: 594,438 || all params: 110,081,292 || trainable%: 0.5400


In [ ]:
qlora_training_args = TrainingArguments(
    output_dir="./results/qlora",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    fp16=False,
    bf16=True,
    push_to_hub=False,
    report_to = None
)

qlora_trainer = Trainer(
    model=peft_model,
    args=qlora_training_args,
    train_dataset=token_data['train'],
    eval_dataset=token_data['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1954078278.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  qlora_trainer = Trainer(


In [ ]:
qlora_trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.892100,0.835388,0.692500
2,0.818800,0.772531,0.719500
3,0.782400,0.756259,0.727500


TrainOutput(global_step=3000, training_loss=0.8538721415201823, metrics={'train_runtime': 1162.7255, 'train_samples_per_second': 41.282, 'train_steps_per_second': 2.58, 'total_flos': 3179359420416000.0, 'train_loss': 0.8538721415201823, 'epoch': 3.0})

In [ ]:
QLORA_SAVE_PATH = "/content/drive/MyDrive/ANLP_weights/qlora_adapter"
qlora_trainer.save_model(QLORA_SAVE_PATH)
print(f"QLoRA adapter saved to {QLORA_SAVE_PATH}")

QLoRA adapter saved to /content/drive/MyDrive/ANLP_weights/qlora_adapter
